<a href="https://colab.research.google.com/github/Nusrahkhan/TSIG/blob/main/Cat_or_Dog(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING**

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

# **LOADING** **DATASET**

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!kaggle datasets download arnaudeq/cats-vs-dogs-1000

Dataset URL: https://www.kaggle.com/datasets/arnaudeq/cats-vs-dogs-1000
License(s): unknown
 94% 167M/177M [00:01<00:00, 116MB/s]
100% 177M/177M [00:01<00:00, 106MB/s]


In [7]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cats-vs-dogs-1000.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_ds = train_datagen.flow_from_directory(
    '/content/dogs_cats_sample_1000/train',
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_ds = train_datagen.flow_from_directory(
    '/content/dogs_cats_sample_1000/valid',
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1600 images belonging to 2 classes.
Found 399 images belonging to 2 classes.


#**CNN** **MODEL**

In [24]:
model = Sequential()

model.add(Conv2D(32, kernel_size =(3, 3), padding = 'same', activation = 'relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size =(3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())


model.add(Conv2D(128, kernel_size =(3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 56, 56, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │      12,845,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,939,457 (49.36 MB)

 Trainable params: 12,939,009 (49.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [26]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [27]:
history = model.fit(train_ds,epochs=16,validation_data=validation_ds)

Epoch 1/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 102ms/step - accuracy: 0.5814 - loss: 7.2156 - val_accuracy: 0.5038 - val_loss: 2.9595
Epoch 2/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.6078 - loss: 1.5333 - val_accuracy: 0.4962 - val_loss: 3.5227
Epoch 3/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 110ms/step - accuracy: 0.6329 - loss: 1.4085 - val_accuracy: 0.4912 - val_loss: 4.2586
Epoch 4/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 0.6022 - loss: 1.3609 - val_accuracy: 0.5138 - val_loss: 5.0743
Epoch 5/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - accuracy: 0.6615 - loss: 1.2737 - val_accuracy: 0.5088 - val_loss: 3.1077
Epoch 6/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.6458 - loss: 1.2494 - val_accuracy: 0.5539 - val_loss: 2.3194
Epoch 7/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 93ms/step - accuracy: 0.6618 - loss: 1.1733 - val_accuracy: 0.5840 - val_loss: 1.5221
Epoch 8/16
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.7015 - loss: 1.1093 - val_accuracy: 0.

In [30]:
model.save('/content/model.keras')

In [31]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    prediction = model.predict(img_array, verbose=0)
    probability = prediction[0][0]

    if probability > 0.5:
        label = "dog"
        confidence = probability
    else:
        label = "cat"
        confidence = 1 - probability

    print(f"\nFinal Results:")
    print(f"Image: {os.path.basename(img_path)}")
    print(f"Prediction: {label}")
    print(f"Confidence: {confidence:.2%}")

    return label, confidence

predict_image('/content/dog.jpg', model)
predict_image('/content/cat3.jpg', model)


Final Results:
Image: dog.jpg
Prediction: dog
Confidence: 99.98%

Final Results:
Image: cat3.jpg
Prediction: cat
Confidence: 93.45%


('cat', 0.9345472753047943)